In [361]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
#from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam
import time

In [362]:
data = pd.read_csv('CNA.csv')


In [363]:
data

,Date,Open,High,Low,Close,Adj Close,Volume
0,1973-02-21,0.000000,5.666667,5.500000,5.500000,3.902713,45900
1,1973-02-22,0.000000,5.583333,5.500000,5.541667,3.932280,48900
2,1973-02-23,0.000000,5.625000,5.375000,5.375000,3.814015,957900
3,1973-02-26,0.000000,5.458333,5.333333,5.458333,3.873148,54900
4,1973-02-27,0.000000,5.500000,5.375000,5.416667,3.843580,44700
...,...,...,...,...,...,...,...
11879,2020-03-26,31.120001,32.770000,30.620001,32.619999,32.619999,260000
11880,2020-03-27,31.600000,32.830002,29.840000,31.990000,31.990000,313400
11881,2020-03-30,32.029999,33.240002,31.150000,32.980000,32.980000,248500
11882,2020-03-31,32.720001,32.720001,30.879999,31.040001,31.040001,267700


In [364]:
data = data.filter(['Close'])


In [365]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,Close
10884,31.990000
10885,31.790001
10886,32.099998
10887,32.099998
10888,32.299999
...,...
11879,32.619999
11880,31.990000
11881,32.980000
11882,31.040001


In [366]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [367]:
dataScale

array([[0.17210466],
       [0.16493367],
       [0.17604868],
       [0.17604868],
       [0.18321974],
       [0.17820004],
       [0.15130868],
       [0.16529217],
       [0.16565072],
       [0.17246322],
       [0.17067049],
       [0.16708496],
       [0.15812118],
       [0.20258155],
       [0.19290064],
       [0.18931518],
       [0.18572959],
       [0.18752232],
       [0.18608821],
       [0.19541049],
       [0.1817855 ],
       [0.18823944],
       [0.17569019],
       [0.17963428],
       [0.16780208],
       [0.18644671],
       [0.18070989],
       [0.1964861 ],
       [0.195052  ],
       [0.21082821],
       [0.20544989],
       [0.19863746],
       [0.20652564],
       [0.20329867],
       [0.20078882],
       [0.19720322],
       [0.18895655],
       [0.18572959],
       [0.18142701],
       [0.1817855 ],
       [0.17891716],
       [0.1767658 ],
       [0.14879883],
       [0.12692717],
       [0.12477588],
       [0.12405876],
       [0.12585156],
       [0.134

In [368]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [369]:
data = train_data

#dataScale = data
train_percent = 0.8
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [370]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 14
size_predict = 1
stepWindow = 1
num_features=1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [371]:
start_time = time.time()

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=4, activation='relu', input_shape=(size_window, num_features)))
model.add(Conv1D(filters=32, kernel_size=4, activation='relu'))

model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [372]:
model.fit(X_train, y_train, epochs = 8000, batch_size = 64)


Epoch 1/8000
10/10 [==============================] - 1s 3ms/step - loss: 0.1987
Epoch 2/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0246
Epoch 3/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0088
Epoch 4/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0046
Epoch 5/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0031
Epoch 6/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0024
Epoch 7/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0021
Epoch 8/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 9/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 10/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0020
Epoch 11/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 12/8000
10/10 [==============================] - 0s 2ms/step - loss: 0.0019
Epoch 13/8000
10/10 [====

10/10 [==============================] - 0s 1ms/step - loss: 6.0845e-04
Epoch 194/8000
10/10 [==============================] - 0s 2ms/step - loss: 7.0590e-04
Epoch 195/8000
10/10 [==============================] - 0s 1ms/step - loss: 6.5029e-04
Epoch 196/8000
10/10 [==============================] - 0s 2ms/step - loss: 6.4215e-04
Epoch 197/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.8928e-04
Epoch 198/8000
10/10 [==============================] - 0s 1ms/step - loss: 5.8141e-04
Epoch 199/8000
10/10 [==============================] - 0s 1ms/step - loss: 5.7600e-04
Epoch 200/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.7266e-04
Epoch 201/8000
10/10 [==============================] - 0s 2ms/step - loss: 6.7893e-04
Epoch 202/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.9987e-04
Epoch 203/8000
10/10 [==============================] - 0s 1ms/step - loss: 6.0344e-04
Epoch 204/8000
10/10 [==============================] - 0s

10/10 [==============================] - 0s 2ms/step - loss: 6.6283e-04
Epoch 382/8000
10/10 [==============================] - 0s 1ms/step - loss: 4.8667e-04
Epoch 383/8000
10/10 [==============================] - 0s 1ms/step - loss: 6.0003e-04
Epoch 384/8000
10/10 [==============================] - 0s 1ms/step - loss: 5.8435e-04
Epoch 385/8000
10/10 [==============================] - 0s 1ms/step - loss: 5.2292e-04
Epoch 386/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.1917e-04
Epoch 387/8000
10/10 [==============================] - 0s 1ms/step - loss: 5.7970e-04
Epoch 388/8000
10/10 [==============================] - 0s 1ms/step - loss: 4.8693e-04
Epoch 389/8000
10/10 [==============================] - 0s 1ms/step - loss: 5.1850e-04
Epoch 390/8000
10/10 [==============================] - 0s 1000us/step - loss: 6.6512e-04
Epoch 391/8000
10/10 [==============================] - 0s 1ms/step - loss: 7.0435e-04
Epoch 392/8000
10/10 [==============================] -

10/10 [==============================] - 0s 1ms/step - loss: 4.6026e-04
Epoch 570/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.3335e-04
Epoch 571/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.3443e-04
Epoch 572/8000
10/10 [==============================] - 0s 985us/step - loss: 4.7460e-04
Epoch 573/8000
10/10 [==============================] - 0s 1ms/step - loss: 4.5747e-04
Epoch 574/8000
10/10 [==============================] - 0s 1ms/step - loss: 4.4062e-04
Epoch 575/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.6525e-04
Epoch 576/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.2953e-04
Epoch 577/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.5377e-04
Epoch 578/8000
10/10 [==============================] - 0s 1ms/step - loss: 4.8144e-04
Epoch 579/8000
10/10 [==============================] - 0s 1ms/step - loss: 4.6700e-04
Epoch 580/8000
10/10 [==============================] - 

10/10 [==============================] - 0s 2ms/step - loss: 4.2232e-04
Epoch 758/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.8539e-04
Epoch 759/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.4231e-04
Epoch 760/8000
10/10 [==============================] - 0s 1ms/step - loss: 4.1905e-04
Epoch 761/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.0820e-04
Epoch 762/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.0566e-04
Epoch 763/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.9713e-04
Epoch 764/8000
10/10 [==============================] - 0s 1ms/step - loss: 4.3056e-04
Epoch 765/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.8550e-04
Epoch 766/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.9220e-04
Epoch 767/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.8915e-04
Epoch 768/8000
10/10 [==============================] - 0s

...

10/10 [==============================] - 0s 2ms/step - loss: 4.0638e-05
Epoch 7084/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.7862e-05
Epoch 7085/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.3876e-05
Epoch 7086/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.6203e-05
Epoch 7087/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.2117e-05
Epoch 7088/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.4050e-05
Epoch 7089/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.9335e-05
Epoch 7090/8000
10/10 [==============================] - 0s 2ms/step - loss: 6.4152e-05
Epoch 7091/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.8244e-05
Epoch 7092/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.4254e-05
Epoch 7093/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.0828e-05
Epoch 7094/8000
10/10 [=========================

10/10 [==============================] - 0s 2ms/step - loss: 5.5234e-05
Epoch 7270/8000
10/10 [==============================] - 0s 2ms/step - loss: 7.1139e-05
Epoch 7271/8000
10/10 [==============================] - 0s 2ms/step - loss: 9.1381e-05
Epoch 7272/8000
10/10 [==============================] - 0s 2ms/step - loss: 9.0614e-05
Epoch 7273/8000
10/10 [==============================] - 0s 2ms/step - loss: 1.4864e-04
Epoch 7274/8000
10/10 [==============================] - 0s 2ms/step - loss: 9.7355e-05
Epoch 7275/8000
10/10 [==============================] - 0s 2ms/step - loss: 7.3981e-05
Epoch 7276/8000
10/10 [==============================] - 0s 2ms/step - loss: 6.0906e-05
Epoch 7277/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.7188e-05
Epoch 7278/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.5275e-05
Epoch 7279/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.1208e-05
Epoch 7280/8000
10/10 [=========================

10/10 [==============================] - 0s 2ms/step - loss: 2.9413e-05
Epoch 7456/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.3824e-05
Epoch 7457/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.1381e-05
Epoch 7458/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.5428e-05
Epoch 7459/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.6436e-05
Epoch 7460/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.6115e-05
Epoch 7461/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.6611e-05
Epoch 7462/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.5542e-05
Epoch 7463/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.3312e-05
Epoch 7464/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.0255e-05
Epoch 7465/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.0931e-05
Epoch 7466/8000
10/10 [=========================

10/10 [==============================] - 0s 2ms/step - loss: 9.7841e-05
Epoch 7642/8000
10/10 [==============================] - 0s 2ms/step - loss: 7.5301e-05
Epoch 7643/8000
10/10 [==============================] - 0s 2ms/step - loss: 6.3596e-05
Epoch 7644/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.8878e-05
Epoch 7645/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.8769e-05
Epoch 7646/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.0147e-05
Epoch 7647/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.4292e-05
Epoch 7648/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.0929e-05
Epoch 7649/8000
10/10 [==============================] - 0s 2ms/step - loss: 2.6897e-05
Epoch 7650/8000
10/10 [==============================] - 0s 2ms/step - loss: 2.7900e-05
Epoch 7651/8000
10/10 [==============================] - 0s 2ms/step - loss: 2.5856e-05
Epoch 7652/8000
10/10 [=========================

10/10 [==============================] - 0s 2ms/step - loss: 3.2424e-05
Epoch 7828/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.7269e-05
Epoch 7829/8000
10/10 [==============================] - 0s 2ms/step - loss: 5.6730e-05
Epoch 7830/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.1894e-05
Epoch 7831/8000
10/10 [==============================] - 0s 2ms/step - loss: 2.6746e-05
Epoch 7832/8000
10/10 [==============================] - 0s 2ms/step - loss: 2.6361e-05
Epoch 7833/8000
10/10 [==============================] - 0s 2ms/step - loss: 2.8813e-05
Epoch 7834/8000
10/10 [==============================] - 0s 2ms/step - loss: 2.5601e-05
Epoch 7835/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.5265e-05
Epoch 7836/8000
10/10 [==============================] - 0s 2ms/step - loss: 4.4507e-05
Epoch 7837/8000
10/10 [==============================] - 0s 2ms/step - loss: 3.8876e-05
Epoch 7838/8000
10/10 [=========================

In [373]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")



5/5 [==============================] - 0s 998us/step
Training time: 146.8798131942749 seconds


In [374]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

mape = mean_absolute_percentage_error(y_test, y_pred)
# In giá trị MSE
print("MSE: ", mse)
print('RMSE: ', rmse)
print('MAPE : ', mape)

MSE:  0.0021808504620269157
RMSE:  0.046699576679311725
MAPE :  0.0531856825249128
